In [1]:
import pandas as pd
from siuba import *
import plydata as pr
import pickle
from plydata.tidy import pivot_wider, pivot_longer

In [3]:
ames_housing = pd.read_csv("data/ames/ames.csv")

In [4]:
(
  ames_housing >>
  mutate(Antique = _.Year_Sold - _.Year_Remod_Add) >>
  mutate(Antique_status = if_else(_.Antique < 10, "new", "old") ) >>
  pr.group_by("Antique_status") >>
  pr.tally()
)
## tally nos ayuda con el conteo es un sininimo de cout

C:\ProgramData\Anaconda3\envs\AMAT\lib\site-packages\plydata\dataframe\common.py:248: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return (gdf for _, gdf in grouper if not gdf.empty)


,Antique_status,n
0,old,1779
1,new,1151


# pivote horizontal - pivot_wider


In [5]:
with open('data/loc_mun_cdmx.pkl', 'rb') as f:
    loc_mun_cdmx = pickle.load(f)

loc_mun_cdmx

,Ambito,NOM_MUN,Total_localidades
0,Rural,Álvaro Obregón,3
1,Rural,La Magdalena Contreras,8
2,Rural,Cuajimalpa de Morelos,14
3,Rural,Tláhuac,31
4,Rural,Xochimilco,78
5,Rural,Tlalpan,95
6,Rural,Milpa Alta,187
7,Urbano,Álvaro Obregón,1
8,Urbano,Azcapotzalco,1
9,Urbano,Benito Juárez,1


In [6]:
(
loc_mun_cdmx >>
    pivot_wider(names_from = "Ambito", values_from = "Total_localidades")
)

,NOM_MUN,Rural,Urbano
0,Azcapotzalco,NaN,1.0
1,Benito Juárez,NaN,1.0
2,Coyoacán,NaN,1.0
3,Cuajimalpa de Morelos,14.0,2.0
4,Cuauhtémoc,NaN,1.0
5,Gustavo A. Madero,NaN,1.0
6,Iztacalco,NaN,1.0
7,Iztapalapa,NaN,1.0
8,La Magdalena Contreras,8.0,1.0
9,Miguel Hidalgo,NaN,1.0


In [7]:
## para rellenar valores
(
loc_mun_cdmx >>
    pivot_wider(
     names_from = "Ambito", 
     values_from = "Total_localidades", 
     values_fill = 0
    )
)

,NOM_MUN,Rural,Urbano
0,Azcapotzalco,0,1
1,Benito Juárez,0,1
2,Coyoacán,0,1
3,Cuajimalpa de Morelos,14,2
4,Cuauhtémoc,0,1
5,Gustavo A. Madero,0,1
6,Iztacalco,0,1
7,Iztapalapa,0,1
8,La Magdalena Contreras,8,1
9,Miguel Hidalgo,0,1


In [9]:
with open('data/us_rent_income.pkl', 'rb') as f:
    us_rent_income = pickle.load(f)

us_rent_income

,GEOID,NAME,variable,estimate,moe
0,01,Alabama,income,24476.0,136.0
1,01,Alabama,rent,747.0,3.0
2,02,Alaska,income,32940.0,508.0
3,02,Alaska,rent,1200.0,13.0
4,04,Arizona,income,27517.0,148.0
...,...,...,...,...,...
99,55,Wisconsin,rent,813.0,3.0
100,56,Wyoming,income,30854.0,342.0
101,56,Wyoming,rent,828.0,11.0
102,72,Puerto Rico,income,NaN,NaN


In [10]:
(
us_rent_income >>
    select(-_.GEOID) >>
    pivot_wider(
     names_from = "variable", 
     values_from = ["estimate", "moe"]
    )
)

,NAME,estimate_income,estimate_rent,moe_income,moe_rent
0,Alabama,24476.0,747.0,136.0,3.0
1,Alaska,32940.0,1200.0,508.0,13.0
2,Arizona,27517.0,972.0,148.0,4.0
3,Arkansas,23789.0,709.0,165.0,5.0
4,California,29454.0,1358.0,109.0,3.0
5,Colorado,32401.0,1125.0,109.0,5.0
6,Connecticut,35326.0,1123.0,195.0,5.0
7,Delaware,31560.0,1076.0,247.0,10.0
8,District of Columbia,43198.0,1424.0,681.0,17.0
9,Florida,25952.0,1077.0,70.0,3.0


# Pivote vertical - pivot_longer

In [11]:
with open('data/relig_income.pkl', 'rb') as f:
    relig_income = pickle.load(f)

relig_income

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27.0,34.0,60.0,81.0,76.0,137.0,122.0,109.0,84.0,96.0
1,Atheist,12.0,27.0,37.0,52.0,35.0,70.0,73.0,59.0,74.0,76.0
2,Buddhist,27.0,21.0,30.0,34.0,33.0,58.0,62.0,39.0,53.0,54.0
3,Catholic,418.0,617.0,732.0,670.0,638.0,1116.0,949.0,792.0,633.0,1489.0
4,Don’t know/refused,15.0,14.0,15.0,11.0,10.0,35.0,21.0,17.0,18.0,116.0
5,Evangelical Prot,575.0,869.0,1064.0,982.0,881.0,1486.0,949.0,723.0,414.0,1529.0
6,Hindu,1.0,9.0,7.0,9.0,11.0,34.0,47.0,48.0,54.0,37.0
7,Historically Black Prot,228.0,244.0,236.0,238.0,197.0,223.0,131.0,81.0,78.0,339.0
8,Jehovah's Witness,20.0,27.0,24.0,24.0,21.0,30.0,15.0,11.0,6.0,37.0
9,Jewish,19.0,19.0,25.0,25.0,30.0,95.0,69.0,87.0,151.0,162.0


In [18]:
(
 relig_income >>
 pivot_longer(
  cols = ['<$10k', '$10-20k', '$20-30k', '$30-40k', '$40-50k', '$50-75k',
          '$75-100k', '$100-150k', '>150k', "Don't know/refused"], 
  names_to = "income", 
  values_to = "count")
)


,religion,income,count
0,Agnostic,<$10k,27.0
1,Atheist,<$10k,12.0
2,Buddhist,<$10k,27.0
3,Catholic,<$10k,418.0
4,Don’t know/refused,<$10k,15.0
...,...,...,...
175,Orthodox,Don't know/refused,73.0
176,Other Christian,Don't know/refused,18.0
177,Other Faiths,Don't know/refused,71.0
178,Other World Religions,Don't know/refused,8.0
